In [1]:
import pandas as pd 
from langchain_chroma import Chroma 
from langchain_community.vectorstores import FAISS

from retriever import Retriever
from evaluator import RetrievalEvaluator

test data

In [2]:
df = pd.read_csv('sample_score.csv') 

In [3]:
df.columns

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'math score', 'reading score',
       'writing score'],
      dtype='object')

In [4]:
df.shape

(20, 8)

### evaluate retriever

In [4]:
modes = ['embed', 'bm25', 'hybrid']
embed_model_names = ["all-MiniLM-L6-v2", "BAAI/LLM-Embedder", "BAAI/bge-small-en-v1.5"]
dbs = [FAISS, Chroma]
db_names = ['faiss', 'chroma']

queries = ["What are the lowest writing scores?", 
           "What are the highest reading scores?", 
           "What are the average math scores?", 
           "If parental level of education has the impact for reading score?", 
           "What's the best comprehensive score?",
           "If food impacts writing score?"]

relevant_docs = [['writing score'], 
                 ['reading score'], 
                 ['math score'], 
                 ['parental level of education','reading score'], 
                 ['writing score','reading score', 'math score'],
                 ['lunch','writing score']]

retriever_results = {'modes': [], 
                     'embed_model_names': [], 
                     'dbs': [], 
                     'Recall@k': [],
                     'MRR': [],
                     'queries': [],
                     'retrieved_docs': []}

for mode in modes:    
    for embed_model_name in embed_model_names:       
        for i in range(len(dbs)):           
            try:
                retriever = Retriever(mode=mode, embed_model_name=embed_model_name, db=dbs[i], top_k=5)
                for j in range(len(queries)):
                    retrieved_doc = retriever.retrieve_schema(queries[j], df, evaluate=True)
                    relevant_doc = relevant_docs[j]
                    evaluator = RetrievalEvaluator(retrieved_doc, relevant_doc)
                    results = evaluator.evaluate()
                    for key, value in results.items():
                        retriever_results[key].append(value)
                        
                    retriever_results['modes'].append(mode)
                    retriever_results['embed_model_names'].append(embed_model_name)
                    retriever_results['dbs'].append(db_names[i])
                    retriever_results['queries'].append(queries[j])
                    retriever_results['retrieved_docs'].append(retrieved_doc)
            except Exception as e:
                print(f"Error: {e}")


Error: Embedding dimension 768 does not match collection dimensionality 384
Error: Embedding dimension 768 does not match collection dimensionality 384


In [5]:
retr_df =pd.DataFrame(retriever_results)

In [6]:
retr_df.groupby(['modes', 'embed_model_names', 'dbs'])[['Recall@k', 'MRR']].agg(
    recall_5_mean=('Recall@k', 'mean'), 
    recall_5_count=('Recall@k', 'count'),
    mrr_mean=('MRR', 'mean'), 
    mrr_count=('MRR', 'count')
)

recall_5_mean  recall_5_count  mrr_mean  \
modes  embed_model_names      dbs                                               
bm25   BAAI/LLM-Embedder      chroma       1.000000               6       1.0   
                              faiss        1.000000               6       1.0   
       BAAI/bge-small-en-v1.5 chroma       1.000000               6       1.0   
                              faiss        1.000000               6       1.0   
       all-MiniLM-L6-v2       chroma       1.000000               6       1.0   
                              faiss        1.000000               6       1.0   
embed  BAAI/LLM-Embedder      faiss        1.000000               6       1.0   
       BAAI/bge-small-en-v1.5 chroma       0.805556               6       1.0   
                              faiss        1.000000               6       1.0   
       all-MiniLM-L6-v2       chroma       0.805556               6       1.0   
                              faiss        1.000000               6       1.0   
hybrid BAAI/LLM-Embedder      faiss        1.000000               6       1.0   
       BAAI/bge-small-en-v1.5 chroma       1.000000               6       1.0   
                              faiss        1.000000               6       1.0   
       all-MiniLM-L6-v2       chroma       1.000000               6       1.0   
                              faiss        1.000000               6       1.0   

                                      mrr_count  
modes  embed_model_names      dbs                
bm25   BAAI/LLM-Embedder      chroma          6  
                              faiss           6  
       BAAI/bge-small-en-v1.5 chroma          6  
                              faiss           6  
       all-MiniLM-L6-v2       chroma          6  
                              faiss           6  
embed  BAAI/LLM-Embedder      faiss           6  
       BAAI/bge-small-en-v1.5 chroma          6  
                              faiss           6  
       all-MiniLM-L6-v2       chroma          6  
                              faiss           6  
hybrid BAAI/LLM-Embedder      faiss           6  
       BAAI/bge-small-en-v1.5 chroma          6  
                              faiss           6  
       all-MiniLM-L6-v2       chroma          6  
                              faiss           6

"embed + Chroma" isn't a good combination choice.


In [7]:

retr_df[retr_df[['Recall@k', 'MRR']].sum(axis=1) < 2]

,modes,embed_model_names,dbs,Recall@k,MRR,queries,retrieved_docs
10,embed,all-MiniLM-L6-v2,chroma,0.333333,1.0,What's the best comprehensive score?,"[reading score, reading score, reading score, ..."
11,embed,all-MiniLM-L6-v2,chroma,0.500000,1.0,If food impacts writing score?,"[writing score, writing score, writing score, ..."
28,embed,BAAI/bge-small-en-v1.5,chroma,0.333333,1.0,What's the best comprehensive score?,"[writing score, writing score, writing score, ..."
29,embed,BAAI/bge-small-en-v1.5,chroma,0.500000,1.0,If food impacts writing score?,"[writing score, writing score, writing score, ..."


In [8]:
retr_df[retr_df[['Recall@k', 'MRR']].sum(axis=1) < 2]['retrieved_docs'].tolist()

[['reading score',
  'reading score',
  'reading score',
  'reading score',
  'reading score'],
 ['writing score',
  'writing score',
  'writing score',
  'writing score',
  'writing score'],
 ['writing score',
  'writing score',
  'writing score',
  'writing score',
  'writing score'],
 ['writing score',
  'writing score',
  'writing score',
  'writing score',
  'writing score']]

Most of the performance is good, just choose one and explain why.

Mode: hybrid (to balance semantic richness and exact matching).

Embedding Model: BAAI/bge-small-en-v1.5 (for a balance between speed and accuracy).

Database: FAISS (for its performance and scalability).

### evaluate code generator



In [5]:
import pandas as pd 
from langchain_community.vectorstores import FAISS

from model import Model
from retriever import Retriever
from prompts import get_prompt, combined_template
from agent import CodeRAGAgent
from evaluator import GenerCodeEvaluator
from execute import extract_code

import warnings
warnings.filterwarnings("ignore")


test data

In [6]:
df = pd.read_csv('sample_score.csv') 

evaluate code generator 

In [3]:
retriever = Retriever(mode='hybrid', embed_model_name="BAAI/bge-small-en-v1.5", db=FAISS, top_k=5)
prompt = get_prompt(combined_template)

In [4]:
model_names = ["llama3.1", "llama-3.3-70b-versatile","mistral", "gemma2-9b-it"]

Kwargs = [{'stable': {'temperature': 0.1, 'top_p': 0.1}},
          {'diverse': {'temperature': 0.9, 'top_p': 0.9}},
          {'default': {}}]

queries = ["What are the two lowest writing scores?", 
           "What are the highest reading scores?", 
           "What are the average math scores?", 
           "If parental level of education has the impact for reading score?", 
           "What's the best comprehensive score?",
           "If food impacts writing score?"]

reference_codes = ["df['writing score'].nsmallest(2)", 
                 "df['reading score'].max()", 
                 "df['math score'].mean()", 
                 "df.groupby('parental level of education')['reading score'].mean()", 
                 "df[['reading score', 'writing score', 'math score']].sum(axis=1).max()",
                 "df.groupby('lunch')['writing score'].mean()"]

code_generator_results = {'model_names': [], 
                         'Exact Match': [],
                         'F1 Score': [],
                         'queries': [],
                         'generateds': [],
                         'param_types': []}


for model_name in model_names:
    for dic_ in Kwargs:
        for param_type, kwargs in dic_.items():
            model = Model(model_name=model_name, **kwargs)
            processor = CodeRAGAgent(retriever, prompt, model, df)
            for i in range(len(queries)):
                try:
                    generated = processor.processor(queries[i])
                    generated_code = extract_code(generated)
                    evaluator = GenerCodeEvaluator(generated_code, reference_codes[i])
                    results = evaluator.evaluate()
                    for key, value in results.items():
                        code_generator_results[key].append(value)
                    
                    code_generator_results['model_names'].append(model_name)
                    code_generator_results['queries'].append(queries[i])
                    code_generator_results['generateds'].append(generated)
                    code_generator_results['param_types'].append(param_type)
                except Exception as e:
                    print(f"Error: {e}")



In [5]:
gen_df = pd.DataFrame(code_generator_results)

In [6]:

gen_df.groupby(['model_names', 'param_types'])[['Exact Match', 'F1 Score']].agg(
    em_mean=('Exact Match', 'mean'), 
    em_count=('Exact Match', 'count'),
    f1_mean=('F1 Score', 'mean'), 
    f1_count=('F1 Score', 'count')
)


em_mean  em_count   f1_mean  f1_count
model_names             param_types                                        
gemma2-9b-it            default      0.833333         6  0.916667         6
                        diverse      0.833333         6  0.916667         6
                        stable       0.833333         6  0.916667         6
llama-3.3-70b-versatile default      1.000000         6  1.000000         6
                        diverse      1.000000         6  1.000000         6
                        stable       1.000000         6  1.000000         6
llama3.1                default      0.833333         6  0.916667         6
                        diverse      0.833333         6  0.916667         6
                        stable       0.833333         6  0.916667         6
mistral                 default      0.666667         6  0.666667         6
                        diverse      0.833333         6  0.916667         6
                        stable       0.666667         6  0.750000         6

llama-3.3-70b-versatile is the best model. diverse is good.


### evaluate interpretation generator

In [1]:
import pandas as pd 
from langchain_community.vectorstores import FAISS

from model import Model
from retriever import Retriever
from prompts import get_prompt, combined_template, interp_template
from agent import CodeRAGAgent, InterpRAGAgent
from evaluator import bert_score_f1
from execute import parse_response

import warnings
warnings.filterwarnings("ignore")


test data




In [2]:
df = pd.read_csv('sample_score.csv') 

evaluate interpretation generator

In [4]:
retriever = Retriever(mode='hybrid', embed_model_name="BAAI/bge-small-en-v1.5", db=FAISS, top_k=5)
prompt = get_prompt(combined_template)
code_model = Model(model_name="gemma2-9b-it",temperature=0.9,top_p=0.9)
processor = CodeRAGAgent(retriever, prompt, code_model, df)
interp_prompt = get_prompt(interp_template)



In [5]:
model_names = ["llama-3.3-70b-versatile","llama3.1", "mistral", "gemma2-9b-it"]

Kwargs = [{'stable': {'temperature': 0.1,'top_p': 0.1}},
          {'diverse': {'temperature': 0.9, 'top_p': 0.9}},
          {'default': {}}]

queries = ["What are the two lowest writing scores?", 
           "What are the highest reading scores?", 
           "What are the average math scores?", 
           "If parental level of education has the impact for reading score?", 
           "What's the best comprehensive score?",
           "If food impacts writing score?"]

references = [["The two lowest writing scores are 22 and 41, associated with indices 11 and 9, respectively, indicating that these are the lowest writing scores in the dataset."], 
              ["The highest reading scores achieved are 86."], 
              ["The calculated result indicates that the average math scores among the given data is approximately 63.85."], 
              ["The relative result indicates that there is a significant difference in reading scores across different levels of parental education, suggesting that parental level of education does have an impact on reading performance."], 
              ["The best comprehensive score is 261, indicating that the sum of reading, writing, and math scores for this particular group or individual is the highest among all available data."],
              ["The relative data suggests that students who received free or reduced lunch had a significantly higher mean writing score (65.7) compared to those who received standard lunch (62.2), indicating a positive impact of food on writing performance."]]

generator_results = {'model_names': [], 
                    'bert_score_f1': [],
                    'queries': [],
                    'responses': [],
                    'param_types': []}

for model_name in model_names:
    for dic_ in Kwargs:
        for param_type, kwargs in dic_.items():
            model = Model(model_name=model_name, **kwargs)
            interp = InterpRAGAgent(interp_prompt, model)
            for i in range(len(queries)):
                try:
                    ctx = processor.invoke(queries[i])
                    answer = interp.processor(ctx, queries[i])
                    response = parse_response(answer)['The concluding response:']
                    f1 = bert_score_f1([response], references[i])
                    generator_results['bert_score_f1'].append(f1)
                    generator_results['model_names'].append(model_name)
                    generator_results['queries'].append(queries[i])
                    generator_results['responses'].append(response)
                    generator_results['param_types'].append(param_type)
                except Exception as e:
                    print(f"Error: {e}")



Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jhnywfrpfrhb5r935e6ewy2h` service tier `on_demand` on : Limit 100000, Used 99717, Requested 1721. Please try again in 20m42.364s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jhnywfrpfrhb5r935e6ewy2h` service tier `on_demand` on : Limit 100000, Used 99717, Requested 1734. Please try again in 20m53.328s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jhnywfrpfrhb5r935e6ewy2h` service tier `on_demand` on : Limit 100000, Used 99717, Requested 1724. Please try again in 20m44.418999

In [11]:
interp_df = pd.DataFrame(generator_results)

In [12]:
interp_df.groupby(['model_names', 'param_types'])[['bert_score_f1']].agg(
    f1_mean=('bert_score_f1', 'mean'), 
    f1_count=('bert_score_f1', 'count')
)



f1_mean  f1_count
model_names  param_types                    
gemma2-9b-it default      0.826000         5
             diverse      0.813333         6
             stable       0.813333         6
llama3.1     default      0.860000         6
             diverse      0.856667         6
             stable       0.890000         6
mistral      default      0.812000         5
             diverse      0.790000         5
             stable       0.822000         5

llama-3.3-70b-versatile is the best model. stable is fine.